In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在Vertex AI上的电信用户流失预测

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/subscriber_churn_prediction/telecom-subscriber-churn-prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
    <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/subscriber_churn_prediction/telecom-subscriber-churn-prediction.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png\" alt="Colab logo"> 在Colab中运行
        </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/workbench/subscriber_churn_prediction/telecom-subscriber-churn-prediction.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本示例演示了在[电信客户流失数据集](https://www.kaggle.com/c/customer-churn-prediction-2020/overview)上构建订阅者流失预测模型。生成的流失模型进一步部署到 Vertex AI 端点，并使用 Vertex AI 的可解释 AI 功能生成解释。

了解更多关于[Vertex AI Workbench](https://cloud.google.com/vertex-ai/docs/workbench/introduction)和[Vertex Explainable AI](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview)。

### 目标

本教程向您展示如何对表格型流失数据集进行探索性数据分析，预处理数据，训练，部署并从流失预测模型中获取预测。本教程的目标如下：

本教程使用以下 Google Cloud ML 服务和资源：

- `Vertex AI Model` 资源
- `Vertex AI Endpoint` 资源
- `Vertex Explainable AI`

执行的步骤包括：

- 从 Cloud 存储路径加载数据
- 执行探索性数据分析（EDA）
- 预处理数据
- 训练一个 scikit-learn 模型
- 评估 scikit-learn 模型
- 将模型保存到 Cloud 存储路径
- 在 Vertex AI 中创建一个模型和一个端点
- 部署训练好的模型到一个端点
- 在托管模型上生成对测试数据的预测和解释
- 关闭模型资源

### 数据集

本教程中使用的数据集是在Kaggle上公开提供的电信客户流失数据集。请参阅[2020年客户流失预测](https://www.kaggle.com/c/customer-churn-prediction-2020/data)。本数据集用于在这个笔记本中使用Vertex AI构建和部署流失预测模型。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

请了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

### 设置您的本地开发环境

**如果您正在使用Colab或者Vertex AI Workbench笔记本**，您的环境已满足运行此笔记本的所有要求。您可以跳过这一步。

**_注意_**：此笔记已在以下环境中经过测试：

* Python版本 = 3.9

**否则**，请确保您的环境满足此笔记本的要求。您需要以下内容：

* Google Cloud SDK

Google Cloud指南中的[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了详细的说明以满足这些要求。以下步骤提供了简洁的指导：

1. [安装并初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端中的命令行上运行 `pip3 install jupyter`。

5. 要启动Jupyter，请在终端中的命令行上运行 `jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装以下所需的包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"
    
! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform \
                                    google-cloud-storage \
                                    category_encoders \
                                    seaborn \
                                    scikit-learn \
                                    pandas \
                                    fsspec \
                                    gcsfs -q 

重新启动内核

在安装了额外的软件包之后，您需要重新启动笔记本内核以便它能找到这些软件包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### 设置您的Google Cloud项目

**无论您使用哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用，可用于支付计算/存储成本。

2. [确保您的项目已启用结算](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter通过以`!`为前缀的行作为shell命令运行，并将以`$`为前缀的Python变量插入这些命令中。

设置您的项目 ID

**如果您不知道您的项目 ID**，可以使用 `gcloud` 来获取您的项目 ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于整个笔记本的操作。以下是Vertex AI支持的区域。建议您选择最接近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶来进行Vertex AI的训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您正在参加实况教程会话，则可能正在使用共享的测试帐户或项目。为了避免所创建资源上的用户之间的名称冲突，您需要为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

###验证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench笔记本**，则您的环境已经经过身份验证。

**如果您正在使用Colab**，请运行下面的单元格，并按提示进行身份验证，通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

1. 在Cloud Console中，转到[**创建服务帐户密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 单击**创建服务帐户**。

3. 在**服务帐户名称**字段中输入名称，然后单击**创建**。

4. 在**授予此服务帐户对项目的访问权限**部分，单击**角色**下拉列表。在过滤框中键入“Vertex AI”，并选择**Vertex AI管理员**。在过滤框中键入“Storage Object Admin”，并选择**Storage Object Admin**。

5. 单击*创建*。包含您密钥的JSON文件将下载到您的本地环境。

6. 在下面的单元格中，将您的服务帐户密钥路径输入为`GOOGLE_APPLICATION_CREDENTIALS`变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS '[your-service-account-key-path]'

### 创建一个云存储桶

**无论您的笔记本环境如何，都需要执行以下步骤。**

在 Vertex AI 中使用 Cloud SDK 创建模型时，您需要提供一个 Cloud Storage 路径，用以保存训练好的模型。
在本教程中，Vertex AI 将训练好的模型保存到一个云存储桶中。使用这个模型构件，您可以创建 Vertex AI 模型和端点资源，以便提供在线预测。

请在下方设置您的 Cloud Storage 桶的名称。它必须在所有 Cloud Storage 桶中是唯一的。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

**只有在您的存储桶不存在时**：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

## 教程

### 导入所需的库

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
import pickle
# configure to don't display the warnings
import warnings

import category_encoders as ce
import seaborn as sns
from google.cloud import aiplatform, storage
from google.cloud.aiplatform_v1.types import SampledShapleyAttribution
from google.cloud.aiplatform_v1.types.explanation import ExplanationParameters
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, plot_roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings("ignore")

### 使用 Pandas 从云存储加载数据

[Kaggle](https://www.kaggle.com/c/customer-churn-prediction-2020/overview) 上的 Telecom-Customer Churn 数据集可在一个公共云存储桶上找到：

```gs://cloud-samples-data/vertex-ai/managed_notebooks/telecom_churn_prediction/train.csv```

使用 Pandas 直接从该 URI 读取数据。

In [ ]:
df = pd.read_csv(
    "gs://cloud-samples-data/vertex-ai/managed_notebooks/telecom_churn_prediction/train.csv"
)
print(df.shape)
df.head()

进行数据探索分析 (EDA)

检查字段的数据类型和空值计数。

In [ ]:
df.info()

当前数据集中没有任何空或空白字段。

检查类别不平衡。

In [ ]:
df["churn"].value_counts(normalize=True)

数据中有14％的流失用户，对于训练流失预测模型来说并不算太糟糕。如果类别不平衡的情况比较严重，可以考虑使用过度取样或欠取样技术来平衡类别分布。

将分类和数值列分开。

In [ ]:
categ_cols = ["state", "area_code", "international_plan", "voice_mail_plan"]
target = "churn"
num_cols = [i for i in df.columns if i not in categ_cols and i != target]
print(len(categ_cols), len(num_cols))

绘制分类列的水平分布图。

In [ ]:
for i in categ_cols:
    df[i].value_counts().plot(kind="bar")
    plt.title(i)
    plt.show()

In [ ]:
print(num_cols)
df["number_vmail_messages"].describe()

检查数值列的分布。

In [ ]:
for i in num_cols:
    # check the Price field's distribution
    _, ax = plt.subplots(1, 2, figsize=(10, 4))
    df[i].plot(kind="box", ax=ax[0])
    df[i].plot(kind="hist", ax=ax[1])
    plt.title(i)
    plt.show()

In [ ]:
# check pairplots for selected features
selected_features = [
    "total_day_calls",
    "total_eve_calls",
    "number_customer_service_calls",
    "number_vmail_messages",
    "account_length",
    "total_day_charge",
    "total_eve_charge",
]
sns.pairplot(df[selected_features])
plt.show()

绘制数字特征的相关矩阵热图。

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(df[num_cols].corr(), annot=True)
plt.show()

### EDA观察

- 在分类字段<b>state</b>中有许多级别/类别。在进一步的步骤中，为这个字段创建一个独热编码向量会大幅增加列数，因此将考虑使用二进制编码技术来编码这个字段。
- 数据中仅有9%的客户拥有国际套餐。
- 仅有少数客户频繁致电客服。
- 只有25%的客户至少拥有16条语音信箱消息，因此`number_vmail_messages`字段的分布存在偏斜。
- 在成对绘图中，大多数特征组合显示出循环模式，表明相应的两个特征之间几乎没有相关性。
- 分钟数和费用之间似乎存在高度相关性。可以丢弃其中一个以避免数据中的多重共线性或冗余特征。

### 预处理数据

删除与高度相关的特征对应的字段。

In [ ]:
drop_cols = [
    "total_day_charge",
    "total_eve_charge",
    "total_night_charge",
    "total_intl_charge",
]
df.drop(columns=drop_cols, inplace=True)
num_cols = list(set(num_cols).difference(set(drop_cols)))
df.shape

对状态特征进行二进制编码（因为有许多级别/类别）。

In [ ]:
encoder = ce.BinaryEncoder(cols=["state"], return_df=True)
data_encoded = encoder.fit_transform(df)
data_encoded.head()

对其余的分类变量进行一独热编码（删除第一级列以避免虚拟变量陷阱情况）。

In [ ]:
def encode_cols(data, col):
    # Creating a dummy variable for the variable 'CategoryID' and dropping the first one.
    categ = pd.get_dummies(data[col], prefix=col, drop_first=True)
    # Adding the results to the master dataframe
    data = pd.concat([data, categ], axis=1)
    return data


for i in categ_cols + [target]:
    if i != "state":
        data_encoded = encode_cols(data_encoded, i)
        data_encoded.drop(columns=[i], inplace=True)

data_encoded.shape

检查数据。

In [ ]:
data_encoded.head()

检查列。

In [ ]:
data_encoded.columns

将数据分成训练集和测试集。

In [ ]:
X = data_encoded[[i for i in data_encoded.columns if i not in ["churn_yes"]]].copy()
y = data_encoded["churn_yes"].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, test_size=0.3, random_state=100
)
print(X_train.shape, X_test.shape)

使用`MinMaxScaler`对数值数据进行缩放。

In [ ]:
sc = MinMaxScaler()
X_train.loc[:, num_cols] = sc.fit_transform(X_train[num_cols])
X_test.loc[:, num_cols] = sc.transform(X_test[num_cols])

使用 scikit-learn 训练一个逻辑回归模型

`class_weight`参数可以调整类别的权重以匹配目标特征。

In [ ]:
model = LogisticRegression(class_weight="balanced")
model = model.fit(X_train, y_train)

## 评估训练好的模型

绘制ROC曲线并显示训练集和测试集上的AUC值

将模型在训练数据上的ROC曲线绘制出来。

In [ ]:
plot_roc_curve(model, X_train, y_train, drop_intermediate=False)
plt.show()

# plot the ROC for the model on test data
plot_roc_curve(model, X_test, y_test, drop_intermediate=False)
plt.show()

确定二元分类的最佳阈值

一般来说，逻辑回归模型输出介于0和1之间的概率分数，需要确定一个阈值来分配类别标签。根据模型的敏感度（真阳性率）和特异性（真阴性率），可以确定最佳阈值。

使用10个不同的概率截断值创建列。

In [ ]:
y_train_pred = model.predict_proba(X_train)[:, 1]
numbers = [float(x) / 10 for x in range(10)]
y_train_pred_df = pd.DataFrame({"true": y_train, "pred": y_train_pred})
for i in numbers:
    y_train_pred_df[i] = y_train_pred_df.pred.map(lambda x: 1 if x > i else 0)

现在计算不同概率截断点下的准确率、灵敏度和特异性。

In [ ]:
cutoff_df = pd.DataFrame(columns=["prob", "accuracy", "sensitivity", "specificity"])

# compute the parameters for each threshold considered
for i in numbers:
    cm1 = confusion_matrix(y_train_pred_df.true, y_train_pred_df[i])
    total1 = sum(sum(cm1))
    accuracy = (cm1[0, 0] + cm1[1, 1]) / total1

    speci = cm1[0, 0] / (cm1[0, 0] + cm1[0, 1])
    sensi = cm1[1, 1] / (cm1[1, 0] + cm1[1, 1])
    cutoff_df.loc[i] = [i, accuracy, sensi, speci]

# Let's plot accuracy sensitivity and specificity for various probabilities.
cutoff_df.plot.line(x="prob", y=["accuracy", "sensitivity", "specificity"])
plt.title("Comparison of performance across various thresholds")
plt.show()

通常情况下，具有平衡敏感度和特异性的模型是首选。在当前情况下，敏感度和特异性曲线相交的阈值可以被认为是最佳阈值。

In [ ]:
threshold = 0.5

# Evaluate train and test sets
y_test_pred = model.predict_proba(X_test)[:, 1]

# to get the performance stats, lets define a handy function


def print_stats(y_true, y_pred):
    # Confusion matrix

    confusion = confusion_matrix(y_true=y_true, y_pred=y_pred)
    print("Confusion Matrix: ")
    print(confusion)

    TP = confusion[1, 1]  # true positive
    TN = confusion[0, 0]  # true negatives
    FP = confusion[0, 1]  # false positives
    FN = confusion[1, 0]  # false negatives

    # Let's see the sensitivity or recall of our logistic regression model
    sensitivity = TP / float(TP + FN)
    print("sensitivity = ", sensitivity)
    # Let us calculate specificity
    specificity = TN / float(TN + FP)
    print("specificity = ", specificity)
    # Calculate false postive rate - predicting conversion when customer didn't convert
    fpr = FP / float(TN + FP)
    print("False positive rate = ", fpr)
    # positive predictive value
    precision = TP / float(TP + FP)
    print("precision = ", precision)
    # accuracy
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    print("accuracy = ", accuracy)
    return


y_train_pred_sm = [1 if i > threshold else 0 for i in y_train_pred]
y_test_pred_sm = [1 if i > threshold else 0 for i in y_test_pred]
# Print the metrics for the model
# on train data
print("Train Data : ")
print_stats(y_train, y_train_pred_sm)
print("\n", "*" * 30, "\n")
# on test data
print("Test Data : ")
print_stats(y_test, y_test_pred_sm)

虽然模型的敏感性和特异性看起来还可以，但精确度可以被认为是低的。这种情况可能在一定程度上是可以接受的，因为从电信行业的商业角度来看，即使意味着有一些非流失客户被错误分类为流失客户，仍然有必要识别出流失客户。

将模型保存到云存储路径

将训练好的模型保存到本地文件`model.pkl`。

In [ ]:
FILE_NAME = "model.pkl"
with open(FILE_NAME, "wb") as file:
    pickle.dump(model, file)

# Upload the saved model file to Cloud Storage
BLOB_PATH = (
    "[your-blob-path]"  # leave blank if no folders inside the bucket are needed.
)

if BLOB_PATH == ("[your-blob-path]"):
    BLOB_PATH = ""

BLOB_NAME = BLOB_PATH + FILE_NAME

bucket = storage.Client().bucket(BUCKET_NAME)
blob = bucket.blob(BLOB_NAME)
blob.upload_from_filename(FILE_NAME)

在Vertex AI中创建支持可解释AI的模型

在创建模型之前，配置模型解释。更多详情，请参阅[在Vertex AI中配置解释](https://cloud.google.com/vertex-ai/docs/explainable-ai/configuring-explanations#scikit-learn-and-xgboost-pre-built-containers)。

为模型资源设置显示名称。

In [ ]:
# Set the model display name
MODEL_DISPLAY_NAME = "[your-model-display-name]"  # @param {type:"string"}

if MODEL_DISPLAY_NAME == "[your-model-display-name]":
    MODEL_DISPLAY_NAME = "subscriber_churn_model"

ARTIFACT_GCS_PATH = f"gs://{BUCKET_NAME}/{BLOB_PATH}"

# Feature-name(Inp_feature) and Output-name(Model_output) can be arbitrary
exp_metadata = {"inputs": {"Inp_feature": {}}, "outputs": {"Model_output": {}}}

In [ ]:
# Create a Vertex AI model resource with support for explanations


aiplatform.init(project=PROJECT_ID, location=REGION)

model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=ARTIFACT_GCS_PATH,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest",
    explanation_metadata=exp_metadata,
    explanation_parameters=ExplanationParameters(
        sampled_shapley_attribution=SampledShapleyAttribution(path_count=25)
    ),
)

model.wait()

print(model.display_name)
print(model.resource_name)

或者，可以使用以下`gcloud`命令来创建模型资源。`explanation-metadata.json`文件包含用于配置模型资源解释的元数据。

```
gcloud beta ai models upload \
  --region=$REGION \
  --display-name=$MODEL_DISPLAY_NAME \
  --container-image-uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest" \
  --artifact-uri=$ARTIFACT_GCS_PATH \
  --explanation-method=sampled-shapley \
  --explanation-path-count=25 \
  --explanation-metadata-file=explanation-metadata.json
```

创建一个端点

In [ ]:
ENDPOINT_DISPLAY_NAME = "[your-endpoint-display-name]"  # @param {type:"string"}
if ENDPOINT_DISPLAY_NAME == "[your-endpoint-display-name]":
    ENDPOINT_DISPLAY_NAME = "subsc_churn_endpoint"

In [ ]:
endpoint = aiplatform.Endpoint.create(
    display_name=ENDPOINT_DISPLAY_NAME, project=PROJECT_ID, location=REGION
)

print(endpoint.display_name)
print(endpoint.resource_name)

### 部署模型到创建的端点

为部署配置部署名称、机器类型和其他部署参数。

In [ ]:
DEPLOYED_MODEL_NAME = "[deployment-model-name]"  # @param {type:"string"}
MACHINE_TYPE = "n1-standard-4"

if DEPLOYED_MODEL_NAME == "[deployment-model-name]":
    DEPLOYED_MODEL_NAME = "subsc_churn_deployment"

In [ ]:
# deploy the model to the endpoint
model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=DEPLOYED_MODEL_NAME,
    machine_type=MACHINE_TYPE,
)

model.wait()

print(model.display_name)
print(model.resource_name)

为了确保模型已部署，可以使用`endpoint.list_models()`方法来检查已部署模型的ID。

In [ ]:
endpoint.list_models()

从部署的模型中获取解释说明##

从托管模型获取测试实例的解释。

In [ ]:
# format a test instance as the request's payload
test_json = [X_test.iloc[0].tolist()]

获取解释并绘制特征归因。

In [ ]:
features = X_train.columns.to_list()


def plot_attributions(attrs):
    """
    Function to plot the features and their attributions for an instance
    """
    rows = {"feature_name": [], "attribution": []}
    for i, val in enumerate(features):
        rows["feature_name"].append(val)
        rows["attribution"].append(attrs["Inp_feature"][i])
    attr_df = pd.DataFrame(rows).set_index("feature_name")
    attr_df.plot(kind="bar")
    plt.show()
    return


def explain_tabular_sample(project: str, location: str, endpoint, instances: list):
    """
    Function to make an explanation request for the specified payload and generate feature attribution plots
    """
    aiplatform.init(project=project, location=location)

    # endpoint = aiplatform.Endpoint(endpoint_id)

    response = endpoint.explain(instances=instances)
    print("#" * 10 + "Explanations" + "#" * 10)
    for explanation in response.explanations:
        print(" explanation")
        # Feature attributions.
        attributions = explanation.attributions

        for attribution in attributions:
            print("  attribution")
            print("   baseline_output_value:", attribution.baseline_output_value)
            print("   instance_output_value:", attribution.instance_output_value)
            print("   output_display_name:", attribution.output_display_name)
            print("   approximation_error:", attribution.approximation_error)
            print("   output_name:", attribution.output_name)
            output_index = attribution.output_index
            for output_index in output_index:
                print("   output_index:", output_index)

            plot_attributions(attribution.feature_attributions)

    print("#" * 10 + "Predictions" + "#" * 10)
    for prediction in response.predictions:
        print(prediction)

    return response


# Get explanations for the test instance
prediction = explain_tabular_sample(PROJECT_ID, REGION, endpoint, test_json)

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：
* Vertex AI模型
* Vertex AI端点
* Cloud Storage存储桶

将`delete_bucket`设置为*True*以删除Cloud Storage存储桶。

In [ ]:
# Undeploy model
endpoint.undeploy_all()

# Delete the endpoint
endpoint.delete()

# Delete the model
model.delete()

# Delete the Cloud Storage bucket
delete_bucket = True
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI